Imports

In [1]:
import os
import openpyxl

Nome dos Arquivos e PATH

In [45]:
Path_Base = "C:\\Users\\tersa\\Documents\\Cesar\\Controle\\TCC\\Tesseract\\Resultados\\"
Base_Dados = "Base_Especifica.xlsx"
Mensagens = "Mensagens.xlsx"

Carrega a Base de Dados

In [46]:
WB_DADOS = openpyxl.load_workbook(Path_Base+Base_Dados)
DADOS = WB_DADOS['Palavras'];

PALAVRAS = [];
NUMEROS = [];

for i in range(2,DADOS.max_row+1):
    PALAVRAS.append(DADOS.cell(row=i,column=1).value);
    NUMEROS.append(DADOS.cell(row=i,column=2).value);
    
WB_DADOS.save('C:\\Users\\tersa\\Documents\\Cesar\\Controle\\TCC\\Tesseract\\Base_Genérica_Alterada.xlsx')

In [47]:
WB_MENSAGENS = openpyxl.load_workbook(Path_Base+Mensagens)
CIFRAS = WB_MENSAGENS['Cifradas']
TRADUCAO = WB_MENSAGENS['Traduzidas']
SALVAR = []
for i in range(2,11):
    CIFRADAS = [];
    coluna = 3;
    while(CIFRAS.cell(row=i,column=coluna).value != '-' and CIFRAS.cell(row=i,column=coluna).value != None):
        CIFRADAS.append(CIFRAS.cell(row=i,column=coluna).value);
        coluna+=1;
    AUX = CIFRAS.cell(row=i,column=2).value;
    ACOES = AUX.replace(' ','').split(';')
    if('FourFigures' in ACOES):
        N_CIFRADAS = getCipher(CIFRADAS, PALAVRAS, NUMEROS, 1);
        for ACT in ACOES:
            if(ACT == '-CTE'):
                N_CIFRADAS = CTE(N_CIFRADAS, PALAVRAS, NUMEROS, 1, -5555);
            if(ACT == '+CTE'):
                N_CIFRADAS = CTE(N_CIFRADAS, PALAVRAS, NUMEROS, 1, 5555);
            if(ACT == 'Rotatethethreelast'):
                N_CIFRADAS = ROTATE(N_CIFRADAS, PALAVRAS, NUMEROS, 1, 3);
            if(ACT == '-n'):
                N_CIFRADAS = subN(N_CIFRADAS, PALAVRAS, NUMEROS, 1);
            if(ACT == 'FourFigures'):
                N_CIFRADAS = REORDER(N_CIFRADAS, PALAVRAS, NUMEROS, 1, 5);
    else:
        N_CIFRADAS = getCipher(CIFRADAS, PALAVRAS, NUMEROS, 0);
        for ACT in ACOES:
            if(ACT == '-CTE'):
                N_CIFRADAS = CTE(N_CIFRADAS, PALAVRAS, NUMEROS, 0, -5555);
            if(ACT == '+CTE'):
                N_CIFRADAS = CTE(N_CIFRADAS, PALAVRAS, NUMEROS, 0, 5555);
            if(ACT == 'Rotatethethreelast'):
                N_CIFRADAS = ROTATE(N_CIFRADAS, PALAVRAS, NUMEROS, 0, 3);
    SALVAR.append(N_CIFRADAS.copy());
    TRADUZIDO = getWords(N_CIFRADAS, PALAVRAS, NUMEROS);
    coluna = 2;
    for word in TRADUZIDO:
        TRADUCAO.cell(row=i,column=coluna).value = word;
        coluna+=1;
        
WB_MENSAGENS.save('C:\\Users\\tersa\\Documents\\Cesar\\Controle\\TCC\\Tesseract\\Resultados\\Mensagens_Base_Específica.xlsx')


Transforma um vetor de palavras em cifras

In [36]:
def getCipher(CIFRADAS, PALAVRAS, NUMEROS, FOUR_FLAG):
    N_CIFRADAS = [];
    for word in CIFRADAS:
        cifra = NUMEROS[PALAVRAS.index(word)];
        if(FOUR_FLAG):
            if(cifra[0] == '0'):
                N_CIFRADAS.append(int(cifra[1:5]))
            else:
                N_CIFRADAS.append(int(cifra[0:4]))
        else:
            N_CIFRADAS.append(int(cifra));
    return N_CIFRADAS

Transforma um vetor de cifras em palavras


In [37]:
def getWords(N_CIFRADAS, PALAVRAS, NUMEROS):
    CIFRADAS = [];
    for cifra in N_CIFRADAS:
        if(COMPLETE_CIPHER(cifra,5) == '00000'):
            CIFRADAS.append('-')
        else:
            word = PALAVRAS[NUMEROS.index(COMPLETE_CIPHER(cifra,5))];
            CIFRADAS.append(word);
    return CIFRADAS

Soma ou Subtrai Constantes das Cifras

In [38]:
def CTE(N_CIFRADAS, PALAVRAS, NUMEROS, FOUR_FLAG, CTE):
    for cifra in N_CIFRADAS:
        AUX = cifra + CTE;
        if(AUX > 25000):
            AUX = AUX - 25000;
        if(AUX < 1):
            AUX = AUX + 25000;
        N_CIFRADAS[N_CIFRADAS.index(cifra)] = AUX;
    return N_CIFRADAS

Rotaciona os últimos N_C caracteres das Cifras

In [39]:
def ROTATE(N_CIFRADAS, PALAVRAS, NUMEROS, FOUR_FLAG, N_C):
    for n_cifra in N_CIFRADAS:
        cifra = COMPLETE_CIPHER(n_cifra,5);
        AUX = '';
        for c in range(0,len(cifra)):
            if(c==len(cifra)-N_C):
                AUX = AUX + cifra[len(cifra)-1];
            elif(c>len(cifra)-N_C):
                AUX = AUX + cifra[c-1];
            else:
                AUX = AUX + cifra[c];
        N_CIFRADAS[N_CIFRADAS.index(n_cifra)] = int(AUX);
    return N_CIFRADAS

Retira -n de cada cifra

In [40]:
def subN(N_CIFRADAS, PALAVRAS, NUMEROS, FOUR_FLAG):
    Qtd = 1;
    for cifra in N_CIFRADAS:
        AUX = cifra - Qtd;
        if(AUX > 25000):
            AUX = AUX - 25000;
        if(AUX < 1):
            AUX = AUX + 25000;
        N_CIFRADAS[N_CIFRADAS.index(cifra)] = AUX;
        Qtd+=1;
    return N_CIFRADAS

Reordena para cifras de N_N números

In [41]:
def REORDER(N_CIFRADAS, PALAVRAS, NUMEROS, FOUR_FLAG, N_N):
    CIFRA = ''
    for n_cifra in N_CIFRADAS:
        cifra = str(n_cifra);
        CIFRA = CIFRA + cifra;
    Qtd = len(CIFRA)//N_N;
    R_N_CIFRADAS = [];
    for i in range(0,Qtd):
        R_N_CIFRADAS.append(int(CIFRA[i*N_N:(i+1)*N_N]));
    return R_N_CIFRADAS

Retorna a Cifra Completa de um Número

In [42]:
def COMPLETE_CIPHER(CIFRA,N):
    N_CIFRA = str(CIFRA)
    if(len(N_CIFRA)<5):
        zeros = N-len(N_CIFRA)
        for i in range(0,zeros):
            N_CIFRA = '0' + N_CIFRA;
    return N_CIFRA

In [2]:
WB = openpyxl.load_workbook('C:\\Users\\tersa\\Documents\\Cesar\\Controle\\TCC\\Tesseract\\Resultados\\Bases.xlsx')
ST_WB = WB['SemTratamento'];
G_WB = WB['Genérica'];
E_WB = WB['Específica'];
B_WB = WB['Base'];

ST = []
ST.append([]);
ST.append([]);
for i in range(2,ST_WB.max_row+1):
    ST[0].append(str(ST_WB.cell(row=i,column=1).value));
    ST[1].append(str(ST_WB.cell(row=i,column=2).value));

G = []
G.append([]);
G.append([]);
for i in range(2,G_WB.max_row+1):
    G[0].append(str(G_WB.cell(row=i,column=1).value));
    G[1].append(str(G_WB.cell(row=i,column=2).value));
    
E = []
E.append([]);
E.append([]);
for i in range(2,E_WB.max_row+1):
    E[0].append(str(E_WB.cell(row=i,column=1).value));
    E[1].append(str(E_WB.cell(row=i,column=2).value));
    
B = []
for i in range(2,B_WB.max_row+1):
    B.append(str(B_WB.cell(row=i,column=1).value));


In [14]:
DIC_ST = 0;
NUM_STG = 0;
NUM_STE = 0;
for i in range(0,len(ST)):
    if(ST[0][i] in B):
        DIC_ST+=1;
    if(ST[0][i] == '' or ST[0][i] == None):
        i_G = G[1].index(ST[1][i]);
        i_E = E[1].index(ST[1][i]);
        if(G[0][i_G] == '' or G[0][i_G] == None):
            NUM_STG+=1;
        if(E[0][i_E] == '' or E[0][i_E] == None):
            NUM_STE+=1;


In [15]:
DIC_G = 0;
NUM_GST = 0;
NUM_GE = 0;
for i in range(0,len(G)):
    if(G[0][i] in B):
        DIC_G+=1;
    if(G[0][i] == '' or G[0][i] == None):
        i_ST = ST[1].index(G[1][i]);
        i_E = E[1].index(G[1][i]);
        if(ST[0][i_ST] == '' or ST[0][i_ST] == None):
            NUM_GST+=1;
        if(E[0][i_E] == '' or E[0][i_E] == None):
            NUM_GE+=1;

In [16]:
DIC_E = 0;
NUM_EST = 0;
NUM_EG = 0;
for i in range(0,len(E)):
    if(E[0][i] in B):
        DIC_E+=1;
    if(E[0][i] == '' or E[0][i] == None):
        i_ST = ST[1].index(E[1][i]);
        i_G = G[1].index(E[1][i]);
        if(ST[0][i_ST] == '' or ST[0][i_ST] == None):
            NUM_EST+=1;
        if(G[0][i_G] == '' or G[0][i_G] == None):
            NUM_EG+=1;

In [19]:
DIC_G

2